In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity =  "all"

In [2]:
import requests as req
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import JSON

# CityBikes

Send a request to CityBikes for the city of your choice. 

In [3]:
# set the data path
data_path = '../data/'

# Set the URL
URL = "https://api.citybik.es/v2/networks"

# Look up the avaliable cities and the networks in the city
# Get the response
response = req.get(URL)
response_json = response.json()
df_networks = pd.json_normalize(data= response_json['networks'])
df_networks.to_csv(data_path + 'city_bikes_networks.csv', index=False)

# 
# print the cities with the most networks
print("The cities with the most networks are:")
df_networks['location.city'].value_counts().head(10)


# set the target city to Vancouver
city = 'Vancouver'
# city = 'Berlin'
# city  = 'Nicosia'
# city  = 'Warszawa'
# city  = 'Ljubljana'
# city  = 'Poznań'
# city  = 'Calais'
# city = 'Toronto'

# print my choice of city
print("My choice of city: ", city)

# lis the networks in the city
network_ids = df_networks[df_networks['location.city'].str.contains(city)]['id'].values[:].tolist()

# print the networks_ids in the city
print("The network(s) in the city: ", network_ids)



The cities with the most networks are:


Nicosia       2
Berlin        2
Warszawa      2
Ljubljana     2
Poznań        2
Calais        2
Bergamo       2
São Paulo     2
Bratislava    2
Budapest      1
Name: location.city, dtype: int64

My choice of city:  Vancouver
The network(s) in the city:  ['mobibikes']


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [4]:
# further request the stations for the target 'city' by using the network_id list:

# create a blank dataframe
df_city_stations = pd.DataFrame()

for id in network_ids:
    URL_stations = "https://api.citybik.es/v2/networks/"+id
    response_stations = req.get(URL_stations)
    response_stations_json = response_stations.json()

    json_dumps_stations = json.dumps(response_stations_json, indent=4)

      
    # append the json_dumps_stations to df_city_stations by using pandas.concat
    
    # works: 
    # df_city_stations = pd.concat([df_city_stations, pd.json_normalize(data= response_stations_json['network']['stations'])], axis=0)
    
    city_bike_stations = req.get("https://api.citybik.es/v2/networks/mobibikes").json()

    # view json in nice format
    # JSON(city_bike_stations)
    df_city_stations_2 = pd.json_normalize(city_bike_stations['network']['stations'])
    
# print the number of stations in the city
print(f"The number of stations in {city} is {df_city_stations_2.shape[0]}.")


The number of stations in Vancouver is 241.


In [5]:
# further request the stations for the target 'city' by using the network_id list:

# create a blank dataframe
df_city_stations = pd.DataFrame()

for id in network_ids:
    URL_stations = "https://api.citybik.es/v2/networks/"+id
    response_stations = req.get(URL_stations)
    response_stations_json = response_stations.json()

    df_city_stations = pd.json_normalize(response_stations_json['network']['stations'])

    
# print the number of stations in the city
print(f"The number of stations in {city} is {df_city_stations.shape[0]}.")

The number of stations in Vancouver is 241.


In [6]:
# examine the columns:
# pd.DataFrame(df_city_stations.columns)
# df_city_stations.columns

In [7]:
# re-order the columns for readability
df_target_city_stations = df_city_stations[['id', 'name', 'latitude', 'longitude', 'empty_slots', 'free_bikes', 'timestamp', 'extra.uid', 'extra.normal_bikes', 'extra.has_ebikes',  'extra.ebikes', 'extra.slots', 'extra.renting', 'extra.returning', 'extra.last_updated']]

# rename the columns: id -> station_id, name -> station_name
df_target_city_stations = df_target_city_stations.rename(columns={'id': 'station_id', 'name': 'station_name'})

Put your parsed results into a DataFrame.

In [8]:
# save  df_city_stations_sub as a csv file
df_target_city_stations.to_csv(data_path + 'target_city_stations_DF.csv', index=False)